In [1]:
import pandas as pd
import seaborn as sns
import os
import glob
import tqdm
import json

In [6]:
import sys
# root = sys.argv[1]
root = '/home/csvt32745/matte/MaskPropagation/vm108_val_midtri_1024x576'
# root = '/home/csvt32745/matte/MaskPropagation/vm240k_val_midtri_512x288'
db_root = './exp_db'
excel_root = './'
dataset = os.path.basename(root)
models = sorted(os.listdir(root))

In [7]:
db_path = os.path.join(db_root, dataset+'.json')
if os.path.isfile(db_path):
    print("Load db: ", db_path)
    with open(db_path) as f:
        db = json.load(f)
    total_metrics_clip = db['avg_clip']
    total_metrics_frame = db['avg_frame']
    if len(total_metrics_clip) > 0:
        metrics = set(total_metrics_clip[next(iter(total_metrics_clip))].keys())
    else:
        metrics = set()
    print(f"{len(total_metrics_clip)} models, {len(metrics)} metrics")
    
    exist_models = set(total_metrics_clip.keys())
    last_time = os.path.getmtime(db_path)
else:
    print("No db exists")
    total_metrics_clip = {}
    total_metrics_frame = {}
    metrics = set()
    exist_models = set()
    last_time = -1

Load db:  ./exp_db/vm108_val_midtri_1024x576.json
96 models, 21 metrics


In [26]:
metric_frame

{'tran_pha_mad': 134.65884397524567,
 'tran_pha_grad': 8.101360275811949,
 'bg_pha_sad': 2812.7801657919404,
 'bg_pha_mse': 6.253909736233574,
 'bg_pha_dtssd': 0.8608206949707112,
 'fg_pha_mad': 148.44460152825403,
 'bg_pha_mad': 10.173908811608367,
 'bg_pha_grad': 0.5779877944309394,
 'pha_sad': 51044.306884295824,
 'fg_pha_mse': 110.50455803692148,
 'pha_conn': 52.21060683484149,
 'fg_pha_grad': 2.9213837707418944,
 'tran_pha_sad': 15578.0652150381,
 'pha_dtssd': 2.5445328545494466,
 'tran_pha_mse': 68.79302396068866,
 'tran_pha_dtssd': 4.964298360147791,
 'pha_mse': 55.11070950190006,
 'pha_grad': 11.600731832334638,
 'fg_pha_sad': 32653.461485115364,
 'pha_mad': 86.54159017655407,
 'fg_pha_dtssd': 1.8728073373684768}

In [23]:
print("Load model metrics...")
is_updated = False
for m in tqdm.tqdm(models):
    if m == 'GT':
        continue
    if not os.path.isfile(p:=os.path.join(root, m, m+'.xlsx')):
        print("Metric file not found: " + m)
        continue
    # if os.path.getmtime(p) < last_time:
    #     print("Models exists: " + m)
    #     continue

    dfs = pd.read_excel(p, sheet_name=None)
    metric_clip = {}
    metric_frame = {}
    keys = set(dfs.keys())
    keys.remove('summary')

    for k in keys:
        df = dfs[k].iloc[1:]
        # print(df)
        if 'vm108' in root:
            df = df.loc[~df[df.columns[1]].str.contains('smokes_')]
        df = df[df.columns[3:]]

        metric_clip[k] = df.mean(axis=1).mean()
        metric_frame[k] = df.mean(axis=0).mean()
        
    total_metrics_clip[m] = metric_clip
    total_metrics_frame[m] = metric_frame
    metrics = metrics | keys
    
    is_updated = True
    break

  0%|          | 0/76 [00:00<?, ?it/s]

Load model metrics...
Metric file not found: RVM
   Unnamed: 0                        Frame-wise avg.  Clip-wise avg.  \
1       vm108                      animal_still_dove               0   
2       vm108                 animal_still_lion_5134               0   
3       vm108                 animal_still_lion_5174               0   
4       vm108                 animal_still_lion_5175               0   
5       vm108       animal_still_origami_spiky_brown               0   
6       vm108        animal_still_origami_spiky_pink               0   
7       vm108                     cloth_cloth_5100_2               0   
8       vm108                     cloth_cloth_5104_2               0   
9       vm108                       cloth_cloth_5106               0   
10      vm108                 cloth_cloth_grail_5152               0   
11      vm108                cloth_cloth_rabbit_5149               0   
12      vm108                cloth_cloth_rabbit_5156               0   
13      vm108  

  3%|▎         | 2/76 [00:12<07:53,  6.40s/it]

   Unnamed: 0                        Frame-wise avg.  Clip-wise avg.  \
1       vm108                      animal_still_dove               0   
2       vm108                 animal_still_lion_5134               0   
3       vm108                 animal_still_lion_5174               0   
4       vm108                 animal_still_lion_5175               0   
5       vm108       animal_still_origami_spiky_brown               0   
6       vm108        animal_still_origami_spiky_pink               0   
7       vm108                     cloth_cloth_5100_2               0   
8       vm108                     cloth_cloth_5104_2               0   
9       vm108                       cloth_cloth_5106               0   
10      vm108                 cloth_cloth_grail_5152               0   
11      vm108                cloth_cloth_rabbit_5149               0   
12      vm108                cloth_cloth_rabbit_5156               0   
13      vm108                full_body_dancing_man_2            

In [17]:
if not is_updated:
    print("No file updated, exit")
    exit()

No file updated, exit


In [18]:
print("Save db: ", db_path)
print(f"{len(total_metrics_clip)} models, {len(metrics)} metrics")
total_dict = {'avg_clip': total_metrics_clip, 'avg_frame': total_metrics_frame}
with open(db_path, 'w') as f:
    json.dump(total_dict, f, indent='\t')

Save db:  ./exp_db/vm240k_val_midtri_512x288.json
9 models, 21 metrics


In [ ]:
excel_path = os.path.join(excel_root, dataset+".xlsx")
print("Save excel: ", excel_path)
with pd.ExcelWriter(excel_path) as writer:  
    for k, df in total_dict.items():
        pd.DataFrame(df).sort_index(ascending=False).to_excel(writer, sheet_name=k)

: 

: 